#  Variational Inference in Stan

Variational inference is a scalable technique for approximate Bayesian inference.
Stan implements an automatic variational inference algorithm,
called Automatic Differentiation Variational Inference (ADVI)
which searches over a family of simple densities to find the best
approximate posterior density.
ADVI produces an estimate of the parameter means together with a sample
from the approximate posterior density.

ADVI approximates the variational objective function, the evidence lower bound or ELBO,
using stochastic gradient ascent.
The algorithm ascends these gradients using an adaptive stepsize sequence
that has one parameter ``eta`` which is adjusted during warmup.
The number of draws used to approximate the ELBO is denoted by ``elbo_samples``. 
ADVI heuristically determines a rolling window over which it computes
the average and the median change of the ELBO.
When this change falls below a threshold, denoted by ``tol_rel_obj``,
the algorithm is considered to have converged.

### Example: variational inference for model ``bernoulli.stan``

In CmdStanPy, the `CmdStanModel` class method `variational` invokes CmdStan with
`method=variational` and returns an estimate of the approximate posterior
mean of all model parameters as well as a set of draws from this approximate
posterior.

In [1]:
import os
from cmdstanpy.model import CmdStanModel
from cmdstanpy.utils import cmdstan_path

bernoulli_dir = os.path.join(cmdstan_path(), 'examples', 'bernoulli')
bernoulli_path = os.path.join(bernoulli_dir, 'bernoulli.stan')
bernoulli_data = os.path.join(bernoulli_dir, 'bernoulli.data.json')
# instantiate, compile bernoulli model
bernoulli_model = CmdStanModel(stan_file=bernoulli_path)
# run CmdStan's variational inference method, returns object `CmdStanVB`
vi = bernoulli_model.variational(data=bernoulli_data)

INFO:cmdstanpy:found newer exe file, not recompiling


INFO:cmdstanpy:compiled model file: /home/nick/.cmdstan/cmdstan-2.27.0/examples/bernoulli/bernoulli


INFO:cmdstanpy:start chain 1


INFO:cmdstanpy:finish chain 1


The class [`CmdStanVB`](https://cmdstanpy.readthedocs.io/en/latest/api.html#stanvariational) provides the following properties to access information about the parameter names, estimated means, and the sample:
 + `column_names`
 + `variational_params_dict`
 + `variational_params_np`
 + `variational_params_pd`
 + `variational_sample`

In [2]:
print(vi.column_names)

('lp__', 'log_p__', 'log_g__', 'theta')


In [3]:
print(vi.variational_params_dict['theta'])

0.23554


In [4]:
print(vi.variational_sample.shape)

(1000, 4)


These estimates are only valid if the algorithm has converged to a good
approximation. When the algorithm fails to do so, the `variational`
method will throw a `RuntimeError`.


In [5]:
fail_model = CmdStanModel(stan_file='eta_should_fail.stan')
vi = fail_model.variational()

INFO:cmdstanpy:compiling stan program, exe file: /tmp/cmdstanpy/docsrc/examples/eta_should_fail


INFO:cmdstanpy:compiler options: stanc_options=None, cpp_options=None


INFO:cmdstanpy:compiled model file: /tmp/cmdstanpy/docsrc/examples/eta_should_fail


INFO:cmdstanpy:start chain 1


INFO:cmdstanpy:finish chain 1


RuntimeError: The algorithm may not have converged.

Unless you set `require_converged=False`:

In [6]:
vi = fail_model.variational(require_converged=False)

INFO:cmdstanpy:start chain 1


INFO:cmdstanpy:finish chain 1


This lets you inspect the output to try to diagnose the issue with the model

In [7]:
vi.variational_params_dict

OrderedDict([('lp__', 0.0),
             ('log_p__', 0.0),
             ('log_g__', 0.0),
             ('mu[1]', 0.0763767),
             ('mu[2]', -0.00697992)])

See the [api docs](https://cmdstanpy.readthedocs.io/en/latest/api.html), section [`CmdStanModel.variational`](https://cmdstanpy.readthedocs.io/en/latest/api.html#cmdstanpy.CmdStanModel.variational) for a full description of all arguments.